In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from geopy.geocoders import Nominatim
import requests, re

In [10]:
data = pd.read_pickle("./data/df_gesamt_10_08.pkl")

# Check data

In [11]:
data.Qid.isna().sum()

0

In [12]:
(data.Preis == 0).sum()

0

In [13]:
def binary_encode(data, column):
    label_mapping = {k: v for v, k in enumerate(data[column].unique())}
    int_encoded = data[[column]].copy().map(label_mapping)

    max_bin_len = int(int_encoded.max()).bit_length()
    ort_binary_encoded = (
        int_encoded.apply(lambda x: format(x, f"0{max_bin_len}b"))
        .str.split("", expand=True)
        .iloc[:, 1:-1]
        .astype(float)
    )

    return ort_binary_encoded

## Check the 'Object' dtypes 

In [14]:
object_columns = data.select_dtypes(include="object").columns

for col in object_columns:
    print(col, "\n", data[col].unique())
    print(data[col].unique().shape, "\n")

Ort 
 ['Adenbüttel' 'Leiferde' 'Wendeburg' ... 'Eitensheim'
 'Rain (Straubing-Bogen)' 'Buchholz (Aller)']
(5465,) 

Au 
 ['222' '025' '089' '368' '252' '400' '135' '118' '336' '400.0' '086' '073'
 '132' '518.0' '399.0' '525.0' '607.0' '085' '076' '220' '222.0' '25.0'
 '89.0' '368.0' '252.0' '135.0' '118.0' '86.0' '73.0' '132.0' '85.0'
 '76.0' '220.0']
(33,) 

Stellenart 
 ['GF']
(1,) 

Stellensubart 
 ['ST' 'EK' 'SE' 'QA' 'BF' 'PH' 'BU' 'CI' 'BX' 'WH' 'BS' 'PP' 'FH' 'SZ'
 'BL' 'CP']
(16,) 

Beleuchtet 
 ['U' 'B']
(2,) 



## Convert some cols to numerical

In [15]:
data["GJ"] = data["GJ"].astype(float, errors="raise")

# drop Stellenart

In [16]:
data = data.drop("Stellenart", axis=1)

# drop Stellen

In [ ]:
data = data.drop("Stellen", axis=1)  # check later

## Drop Au (the information is in the column Eigenfläche)

In [ ]:
data["Eigenfläche"] = data["Eigenfläche"].astype(float)

In [ ]:
data = data.drop("Au", axis=1)

# One Hot Encode  Beleuchtet cols

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from joblib import dump

In [ ]:
beleuchtet_encoder = OneHotEncoder(sparse_output=False, drop="first")

In [ ]:
data["Beleuchtet"] = beleuchtet_encoder.fit_transform(data[["Beleuchtet"]])

In [ ]:
dump(beleuchtet_encoder, "./data/encoders/beleuchtet_encoder.joblib");

# Encode the Stellensubart col

In [ ]:
encoded_stellensubart = binary_encode(data, "Stellensubart")

In [ ]:
data[
    ["Stellensubart_1", "Stellensubart_2", "Stellensubart_3", "Stellensubart_4"]
] = encoded_stellensubart

In [ ]:
data.drop(columns="Stellensubart", inplace=True)

# Handle missng values of Einwohner

In [ ]:
def get_population(city_name):
    url = f"https://en.wikipedia.org/wiki/{city_name}"

    response = requests.get(url)
    source = response.text

    result = re.findall("""<td class="infobox-data">(\d+,\d+)</td>""", source)

    if len(result):
        return float(result[0].replace(",", ""))
    else:
        return None

In [ ]:
data["Einwohner"] = data.groupby(["Ort", "GJ"])["Einwohner"].transform(
    lambda group: group.ffill().bfill()
)

In [ ]:
missing_city_names = data[data.Einwohner.isna()].Ort.unique()

In [ ]:
result_map = {}

for city_name in missing_city_names:
    population = get_population(city_name)
    result_map[city_name] = population

result_map["Wörthsee"] = 0  # It is a sea
result_map["Winterbach (Rems-Murr-Kreis)"] = 7671
result_map["Lohmen (Sächsische Schweiz-Osterzgebirge)"] = 3112
result_map["Bruck i.d.OPf."] = 4524
result_map["Anger"] = 4526
result_map["Hillesheim (Mainz-Bingen)"] = 672
result_map["Schömberg (Calw)"] = 8068
result_map["Steinach (Sonneberg)"] = 3688
result_map["Steinberg (Vogtlandkreis)"] = 2681
result_map["Plate"] = 3312
result_map["Eisenach (Eisenach)"] = 41806

In [ ]:
for key, value in result_map.items():
    idx = data[data.Ort == key].index
    data.loc[idx, "Einwohner"] = value

In [ ]:
data.Einwohner.isna().sum()

## Handle the missing values of PLZ code using geopy

In [ ]:
data["PLZ"] = data.groupby("Qid")["PLZ"].transform(lambda group: group.ffill().bfill())

In [ ]:
geo_locator = Nominatim(user_agent="geoapiExercises")

In [ ]:
unknown_plz = data[data["PLZ"].isna()]

for index, row in tqdm(unknown_plz.iterrows(), total=len(unknown_plz)):
    coords = f"""{row["Breite"]},{row["Laenge"]}"""
    location = geo_locator.geocode(coords)
    data.loc[index, "PLZ"] = float(location[0].split(",")[-2].strip())

### Check those rows that have "0" PLZ and correct

In [ ]:
zero_plz = data[data["PLZ"] == 0]

for index, row in tqdm(zero_plz.iterrows(), total=len(zero_plz)):
    coords = f"""{row["Breite"]},{row["Laenge"]}"""
    location = geo_locator.geocode(coords)
    data.loc[index, "PLZ"] = float(location[0].split(",")[-2].strip())

In [ ]:
data["PLZ"] = data["PLZ"].astype(float)

# Handle the missing values of Ort code using geopy

In [ ]:
missing_cities = data[data["Ort"].isna()]

for index, row in tqdm(missing_cities.iterrows()):
    coords = f"""{row["Breite"]},{row["Laenge"]}"""
    location = geo_locator.geocode(coords)
    data.loc[index, "Ort"] = location[0].split(",")[2].strip()

# Handle missting values of PPSVACWert

In [ ]:
data["PPSVACWert"] = data.groupby("Qid")["PPSVACWert"].transform(
    lambda group: group.ffill().bfill()
)

In [ ]:
data["PPSVACWert"] = data["PPSVACWert"].fillna(0)

## Drop outliers

In [ ]:
outlier_indices = data[data["PPSVACWert"] == 16082012.0].index

In [ ]:
data.drop(outlier_indices, inplace=True)

# Handle missing values of $T_{i}$

In [ ]:
data[[f"T{i}" for i in range(1, 35)]] = data[[f"T{i}" for i in range(1, 35)]].fillna(-1)

# Handle missting values of Preis

In [ ]:
old_data = pd.read_pickle("./data/df_gesamt.pkl")

In [ ]:
missing_qids = data[data.Preis.isna()][["Qid", "GJ"]]

for qid, year in missing_qids.values:
    row = old_data[(old_data.Qid == qid) & (old_data.GJ == year)]

    if row.shape[0]:
        data.loc[row.index, "Preis"] = row["Preis"]

In [ ]:
data["Preis"] = data.groupby("Qid")["Preis"].transform(
    lambda group: group.ffill().bfill()
)

In [ ]:
data["Preis"] = data["Preis"].fillna(-1)

# Save the data

In [ ]:
# reset the indicex
data.reset_index(drop=True, inplace=True)

In [ ]:
data.to_pickle("./data/df_gesamt_15_08_prepocessed_einworner_added.pkl")